In [43]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% ! important; }<style>"))

In [44]:
import sys
import pyvisa as visa
import time
import struct
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

In [45]:
#connect to scope
try:
    SCOPE_VISA_ADDRESS = 'USB0::0x0957::0x1790::MY58031279::0::INSTR' #VISA address goes here, can be obtained from 
    # IO Libraries Connection Expert or by using the scope panel: utilities -> configure LAN -> USB VISA Address 
    rm = visa.ResourceManager()
    scope = rm.open_resource(SCOPE_VISA_ADDRESS)
    print(scope.query('*IDN?'))
    
except Exception:
    print("Unable to connect to oscilloscope at " + str(SCOPE_VISA_ADDRESS) + ". Aborting script.")
    sys.exit()

AGILENT TECHNOLOGIES,MSO-X 6004A,MY58031279,07.30.2019051434



In [46]:
# establish number of points and time out
USER_REQUESTED_POINTS = 1000
GLOBAL_TOUT =  10000 # IO time out in milliseconds

In [47]:
## Save Locations
BASE_FILE_NAME = "my_data"
BASE_DIRECTORY = "C:\\Users\\Public\\"
    ## IMPORTANT NOTE:  This script WILL overwrite previously saved files!

In [48]:
#connect to visa library that contains code and data that can be used by more than one program at the same time
rm = visa.ResourceManager('C:\\Windows\\System32\\visa32.dll') # this uses PyVisa
## This directory will need to be changed if VISA was installed somewhere else.
# it is generally not needed to call it explicitly: rm = visa.ResourceManager()

In [49]:
# Clear the status data structures, the device-defined error queue, and the Request-for-OPC flag
scope.clear()
## DO NOT RESET THE SCOPE! - since that would wipe out data...

## Data should already be acquired and scope should be STOPPED (Run/Stop button is red).

In [50]:
## Determine Which channels are on AND have acquired data - Scope should have already acquired data and be in a stopped 
# state (Run/Stop button is red).

## Get Number of analog channels on scope
IDN = str(scope.query("*IDN?"))
IDN = IDN.split(',') # IDN parts are separated by commas, so parse on the commas
MODEL = IDN[1]
NUMBER_ANALOG_CHS = int(MODEL[len(MODEL)-2])
print("There are " + str(NUMBER_ANALOG_CHS) + " analog channels on this device")

There are 4 analog channels on this device


In [51]:
CHS_LIST = np.zeros(NUMBER_ANALOG_CHS) # Create empty array to store channel states
NUMBER_CHANNELS_ON = 0
print(CHS_LIST)

[0. 0. 0. 0.]


In [55]:
scope.write(":WAVeform:POINts:MODE MAX") # MAX mode works for all acquisition types, so this is done here to avoid Acq. Type vs points mode problems. Adjusted later for specific acquisition types.


for i in range(NUMBER_ANALOG_CHS):
    ch = i # Channel number
    On_Off = int(scope.write(":CHANnel{}:DISPlay?".format(ch))) # checks if the channel is displayed by returning a 1 or 0
    if On_Off == 1:
        print("channel " + str(i+1) + " is on")
    else:
        print("channl" + str(i+1) + " is off")

channl1 is off
channl2 is off
channl3 is off
channl4 is off
